In [ ]:
import pandas as pd
import sqlite3
import os

In [ ]:
db = sqlite3.connect(r"C:/Users/DH/Desktop/Desafio.db")

In [ ]:
# Quantidade Faculdades Publicas e Particular por Estado

Query1 = ('''

SELECT  T1.CO_UF,
        T1.TP_CATEGORIA_ADMINISTRATIVA,
        SUM(CASE WHEN T1.TP_CATEGORIA_ADMINISTRATIVA IN (1,2,3) THEN 1 ELSE 0 END) AS QTD_PUBLICA,
        SUM(CASE WHEN T1.TP_CATEGORIA_ADMINISTRATIVA IN (4,5,6) THEN 1 ELSE 0 END) AS QTD_PARTICULAR
FROM    DM_IES T1
GROUP BY T1.CO_UF, T1.TP_CATEGORIA_ADMINISTRATIVA;

              ''')

df_1 = pd.read_sql_query(Query1,db)
df_1

In [ ]:
# Quantidade Alunos por Instituição de Ensino Superior

Query2 = ('''

SELECT  T2.NO_IES,
        COUNT(T1.CO_ALUNO) AS QTD_ALUNOS
FROM DM_ALUNO T1
INNER JOIN DM_IES T2 ON T1.CO_IES = T2.CO_IES
GROUP BY T2.NO_IES;

              ''')

df_2 = pd.read_sql_query(Query2,db)
df_2

In [ ]:
# Menos mulheres nos cursos de exatas?

Query3 = ('''

    SELECT CASE WHEN T3.CO_OCDE_AREA_GERAL = 1 THEN 'Educação'
        WHEN T3.CO_OCDE_AREA_GERAL = 2 THEN 'Humanidades e Artes'
        WHEN T3.CO_OCDE_AREA_GERAL = 3 THEN 'Ciências Sociais, Negócios e Direito'
        WHEN T3.CO_OCDE_AREA_GERAL = 4 THEN 'Ciências, Matemática e Computação'
        WHEN T3.CO_OCDE_AREA_GERAL = 5 THEN 'Engenharia, Produção e Construção'
        WHEN T3.CO_OCDE_AREA_GERAL = 6 THEN 'Agricultura e Veterinária'
        WHEN T3.CO_OCDE_AREA_GERAL = 7 THEN 'Saúde e Bem Estar Social'
        WHEN T3.CO_OCDE_AREA_GERAL = 8 THEN 'Serviços'
        END AS AREA,
        SUM(CASE WHEN T1.TP_SEXO = 1 THEN 1 ELSE 0 END) AS QTD_FEM,
        SUM(CASE WHEN T1.TP_SEXO = 2 THEN 1 ELSE 0 END) AS QTD_MASC,
        SUM(CASE WHEN T1.TP_SEXO = 1 THEN 1 ELSE 0 END)/SUM(1)*100 AS PERC_FEM,
        SUM(CASE WHEN T1.TP_SEXO = 2 THEN 1 ELSE 0 END)/SUM(1)*100 AS PERC_MASC
    FROM DM_ALUNO T1
    INNER JOIN DM_IES T2 ON T1.CO_IES = T2.CO_IES
    INNER JOIN DM_CURSO T3 ON T1.CO_CURSO = T3.CO_CURSO
    WHERE T3.CO_OCDE_AREA_GERAL IS NOT NULL
    GROUP BY T3.CO_OCDE_AREA_GERAL;

              ''')

df_3 = pd.read_sql_query(Query3,db)
df_3

In [ ]:
# Quantidade de cursos que abrem ao ano

Query4 = ('''

    SELECT ANO, SUM(QTD_CURSO) AS QTD_CURSO
    FROM
                    (SELECT YEAR(T1.DT_INICIO_FUNCIONAMENTO) AS ANO,
                    1 AS QTD_CURSO
                    FROM DM_CURSO T1
                    WHERE T1.DT_INICIO_FUNCIONAMENTO IS NOT NULL)
    GROUP BY ANO;

              ''')

df_4 = pd.read_sql_query(Query4,db)
df_4

In [ ]:
# Taxa de concluientes foi utilizado a qtde de concluientes pela quantidade de matriculados por curso
# As taxas de dificuldade se mantém consistentes entre 13 a 18% sem grandes variações
# A dificuldade de se analisar dos últimos meses é pelo fato dos matriculados ainda estarem com o curso em andamento

Query5 = ('''

    SELECT ANO, SUM(QT_MATRICULA_TOTAL) AS QT_MATRICULA_TOTAL, 
            SUM(QT_CONCLUINTE_TOTAL) AS QT_CONCLUINTE_TOTAL,
            SUM(QT_CONCLUINTE_TOTAL)/SUM(QT_MATRICULA_TOTAL) AS PERC_CONCLUSAO
        FROM
            (SELECT YEAR(T1.DT_INICIO_FUNCIONAMENTO) AS ANO,
             QT_MATRICULA_TOTAL,
             QT_CONCLUINTE_TOTAL
            FROM DM_CURSO T1
            WHERE T1.DT_INICIO_FUNCIONAMENTO IS NOT NULL)
        GROUP BY ANO;

              ''')

df_5 = pd.read_sql_query(Query5,db)
df_5

In [ ]:
# Realizar no phyton a diferença entre a qtde de matriculados do menor ano pelo o de maior ano

Query6 = ('''

    SELECT CO_UF, ANO, SUM(QT_MATRICULA_TOTAL) AS QT_MATRICULA_TOTAL
        FROM
        (SELECT CO_UF, 
            YEAR(T1.DT_INICIO_FUNCIONAMENTO) AS ANO,
            QT_MATRICULA_TOTAL
            FROM DM_CURSO T1
            WHERE T1.DT_INICIO_FUNCIONAMENTO IS NOT NULL
            AND CO_UF IS NOT NULL)
        GROUP BY CO_UF, ANO;

              ''')

df_6 = pd.read_sql_query(Query6,db)
df_6